# Day 8: Hugging Face Trainer API 进阶

**Trainer** 是 Hugging Face 提供的高层训练框架，**大幅简化了模型微调的代码量**。它自动处理分布式训练、混合精度训练、梯度积累等复杂逻辑。

**学习目标：**
- 掌握 `Trainer` 的完整微调流程（数据准备 → 模型加载 → 训练 → 评估）
- 理解 `DataCollatorWithPadding` 的动态填充机制
- 学习 `TrainingArguments` 的关键参数配置
- 实现自定义评估指标函数

**核心组件：**
1. **TrainingArguments**：配置所有训练超参数
2. **Trainer**：封装训练、验证、保存等逻辑
3. **DataCollator**：动态填充与批处理
4. **evaluate**：加载和计算评估指标

**提示**：Trainer 是生产级工具，但也可以直接使用原始 PyTorch 训练循环以获得更多控制。

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments,Trainer,AutoModelForSequenceClassification,get_scheduler
import numpy as np
import evaluate
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

## GLUE 基准测试

**GLUE（General Language Understanding Evaluation）** 包含 9 个自然语言理解任务：

**单句任务：**
- **CoLA**：语法正确性判断
- **SST-2**：情感分析

**句子对相似度任务：**
- **MRPC**：释义检测
- **STS-B**：语义相似度打分
- **QQP**：问题对等价性

**自然语言推理（NLI）任务：**
- **MNLI**：蕴含关系判断
- **QNLI**：问题-句子匹配
- **RTE**：文本蕴含
- **WNLI**：代词指代解析

**本讲义采用 MRPC 任务** —— 判断两个句子是否为同义句，是一个经典的句子对分类任务。

## 数据预处理与 Hugging Face datasets API

**datasets.load_dataset()**：高效加载数据
- 自动从 Hub 下载并缓存
- 基于 Apache Arrow，支持流式处理
- 支持切片、过滤、映射等高级操作

**map() 方法**：对数据集应用转换
- `batched=True`：批量处理，加速速度 5-10 倍
- `remove_columns`：删除不需要的列
- `desc`：显示进度条

**set_format()**：转换数据格式
- `type="torch"`：转换为 PyTorch 张量
- `columns`：指定转换的列

这种设计使得即使数据集超过内存容量，也能高效处理。

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

**load_dataset()**

1. **加载数据集**：
    - 使用 load_dataset() 从 Hub 或本地路径加载数据集。
    - 数据集通常分为 train、validation 和 test 三个拆分（split）。
2. **访问数据**：
    - 数据以类似字典的形式存储，支持切片、迭代和列访问。
    - 每个样本是一个字典，键对应特征（如 text、label）。
3. **处理数据**：
    - 支持映射（map）、过滤（filter）、分片（shard）等操作。
    - 可以无缝与 transformers 的 tokenizer 结合。
4. **输出**：
    - 数据可以直接访问，或转换为 PyTorch/TensorFlow 张量。

代码示例

In [ ]:
# 1. 加载数据集
dataset = load_dataset(
    "emotion",                  # 数据集名称
    split="train",              # 指定拆分（可选：train/validation/test）
    download_mode="reuse_dataset_if_exists",  # 复用已下载的数据
)

# 2. 查看数据集基本信息
print("数据集信息:", dataset)
print("特征:", dataset.features)
print("样本数量:", len(dataset))
print("第一个样本:", dataset[0])

# 3. 访问和切片数据
sample_text = dataset["text"][:3]  # 前 3 个样本的文本
sample_labels = dataset["label"][:3]  # 前 3 个样本的标签
print("\n前 3 个样本文本:", sample_text)
print("前 3 个样本标签:", sample_labels)

# 4. 结合 tokenizer 预处理
# 该函数接收一个字典（与 dataset 的项类似）并返回一个包含 id(input_ids) ，(attention_mask) 和 token_type_ids 键的新字典
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

# 应用映射函数
# Datasets 库中的数据集是以 Apache Arrow 格式存储在磁盘上的，因此你只需将接下来要用的数据加载在内存中，而不是加载整个数据集
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,              # 批量处理
    remove_columns=["text"],   # 删除原始文本列
    desc="Tokenizing dataset"  # 进度条描述
)
print("\n预处理后的第一个样本:", tokenized_dataset[0])

# 5. 过滤数据（示例：只保留标签为 0 的样本）
filtered_dataset = dataset.filter(
    lambda example: example["label"] == 0,
    desc="Filtering for label 0"
)
print("\n过滤后样本数量:", len(filtered_dataset))
print("过滤后第一个样本:", filtered_dataset[0])

# 6. 转换为 PyTorch 格式
tokenized_dataset.set_format(
    type="torch",              # 转换为 PyTorch 张量
    columns=["input_ids", "attention_mask", "label"]
)
print("\n转换为 PyTorch 后的第一个样本:", tokenized_dataset[0])

# 7. 批量访问（示例：取前 2 个样本）
batch = tokenized_dataset[:2]
print("\n批量数据 (input_ids):", batch["input_ids"].shape)
print("批量数据 (labels):", batch["label"])

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

...

数据集信息: Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})
特征: {'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}
样本数量: 16000
第一个样本: {'text': 'i didnt feel humiliated', 'label': 0}

前 3 个样本文本: ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong']
前 3 个样本标签: [0, 0, 3]


Tokenizing dataset:   0%|          | 0/16000 [00:00<?, ? examples/s]


预处理后的第一个样本: {'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Filtering for label 0:   0%|          | 0/16000 [00:00<?, ? examples/s]


过滤后样本数量: 4666
过滤后第一个样本: {'text': 'i didnt feel humiliated', 'label': 0}

转换为 PyTorch 后的第一个样本: {'label': tensor(0), 'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

**输入参数**

1. **load_dataset() 参数**：
    - "emotion": 数据集名称。
    - split: 指定加载的拆分（可选，默认加载所有拆分）。
    - cache_dir: 自定义缓存路径,默认路径~/.cache/huggingface/datasets。
    - download_mode: 控制下载行为（"reuse_dataset_if_exists" 复用已下载数据）。
2. **map() 参数**：
    - function: 自定义处理函数（如分词）。
    - batched=True: 批量处理，提高效率。
    - remove_columns: 删除不需要的列（如原始 text）。
3. **filter() 参数**：
    - function: 过滤条件（返回布尔值）。
    - desc: 进度条描述。
4. **set_format() 参数**：
    - type: 输出格式（"torch", "tensorflow", "numpy", 或 None）。
    - columns: 指定转换的列。

**输出内容**

1. **原始数据集**：
    - 一个 Dataset 对象，包含特征和样本。
    - 访问方式类似字典或列表。
2. **预处理后数据集**：
    - 包含 tokenizer 输出（如 input_ids, attention_mask）。
    - 移除原始列后更适合模型输入。
3. **过滤后数据集**：
    - 仅保留满足条件的样本。
4. **PyTorch 格式**：
    - 数据转换为张量，直接可用于训练。

**批量访问**

- 切片（如 [:2]）返回一个字典，键对应列名，值是张量。
- 形状反映批量大小（batch_size）和序列长度。

**保存数据集**：python

```python
tokenized_dataset.save_to_disk("./my_dataset")
```

**动态填充**

将所有示例填充到该 batch 中最长元素的长度

```python
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
```

## Trainer API

准备数据示例

## DataCollatorWithPadding 详解

**传统 padding**：所有样本填充到最大长度（浪费计算）
- 例如：[10, 100, 50] 序列长度，全部填充到 100，浪费 40+50=90 个 token

**动态 padding**：每个 batch 内填充到该 batch 的最长长度（高效）
- 不同 batch 可能有不同的最大长度
- 平均处理长度更短，计算更快

**DataCollatorWithPadding 的工作原理：**
1. 接收一个 batch 的样本（长度不一）
2. 查找 batch 内的最长长度
3. 将所有样本填充到该长度
4. 返回张量化的批处理数据

**效果**：在保持模型准确性的前提下，通常能提升 20-30% 的训练速度。

In [2]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

## TrainingArguments：训练配置中心

**TrainingArguments** 是一个数据类，集中管理所有训练相关配置。主要参数分类：

**基本设置：**
- `output_dir`：模型和检查点保存目录
- `num_train_epochs`：训练轮数

**批处理与优化：**
- `per_device_train_batch_size`：每个 GPU/CPU 的训练 batch 大小
- `per_device_eval_batch_size`：评估 batch 大小
- `learning_rate`：初始学习率
- `weight_decay`：L2 正则化系数

**日志与保存：**
- `logging_steps`：每隔多少步记录一次日志
- `save_strategy`：保存策略（`"steps"` 或 `"epoch"`）
- `evaluation_strategy`：评估策略（`"no"`、`"steps"` 或 `"epoch"`）

**高级功能：**
- `load_best_model_at_end`：训练结束时加载最佳模型
- `metric_for_best_model`：用于选择最佳模型的指标
- `fp16`：是否使用半精度训练（节省显存）
- `seed`：随机种子，保证可复现性

**推荐配置：**
- 设置 `evaluation_strategy="epoch"` 以在每个 epoch 后评估
- 设置 `save_strategy="epoch"` 和 `load_best_model_at_end=True` 以保存最佳模型
- 设置 `metric_for_best_model="accuracy"` 指定评估指标

**常用参数**

- `output_dir`：模型和检查点保存目录。**默认**：`"./results"`  
- `overwrite_output_dir`：是否覆盖输出目录内容。**默认**：`False`  
- `num_train_epochs`：训练轮数（epoch）。**默认**：`3.0`  
- `per_device_train_batch_size`：每个设备的训练 batch 大小。**默认**：`8`  
- `per_device_eval_batch_size`：每个设备的评估 batch 大小。**默认**：`8`  
- `learning_rate`：初始学习率。**默认**：`5e-5`  
- `weight_decay`：权重衰减（L2 正则）。**默认**：`0.0`  
- `logging_dir`：日志保存目录。**默认**：`"runs/**CURRENT_DATETIME_HOSTNAME**"`  
- `logging_steps`：日志记录步数间隔。**默认**：`500`  
- `evaluation_strategy`：评估策略。**可选值**：`"no"`、`"steps"`、`"epoch"`，**默认**：`"no"`  
- `save_strategy`：模型保存策略。**可选值**：`"no"`、`"steps"`、`"epoch"`，**默认**：`"steps"`  
- `save_steps`：保存模型的步数间隔。**默认**：`500`  
- `seed`：随机种子，保证实验可复现。**默认**：`42`  
- `fp16`：是否使用半精度训练（需 GPU 支持）。**默认**：`False`  
- `load_best_model_at_end`：训练结束时加载最佳模型。**默认**：`False`  
- `metric_for_best_model`：用于选择最佳模型的评估指标。**默认**：`None`（需指定评估指标名称，如 `"accuracy"`）

In [ ]:
## 数据准备（手动训练循环版本）

如果不使用 Trainer，可以手动构造数据加载器：

**必要步骤：**
1. `remove_columns()`：删除不需要的列（如原始句子）
2. `rename_column()`：重命名 label 为 labels（模型要求）
3. `set_format("torch")`：转换为 PyTorch 张量
4. `DataLoader(..., collate_fn=data_collator)`：创建数据加载器

这种方式提供更多控制，适合自定义训练逻辑。

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


定义一个 `Trainer` 把到目前为止构建的所有对象 —— `model` ，`training_args`，训练和验证数据集， `data_collator` 和 `tokenizer` 传递给 `Trainer`

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Trainer 的职责

**Trainer 类** 是一个"训练管家"，自动处理：

**核心流程：**
- 数据分批加载
- 前向传播 & 损失计算
- 反向传播 & 梯度更新
- 学习率调度

**高级功能：**
- 分布式训练（多 GPU/多节点）
- 梯度积累
- 混合精度训练（FP16）
- 梯度裁剪
- 周期性评估与保存最佳模型
- 集成 TensorBoard 日志

**内部机制：**
1. 初始化时接收模型、训练数据、评估数据等
2. 在 `train()` 方法中执行训练循环
3. 在 `evaluate()` 方法中进行评估
4. 自动管理检查点与最佳模型

**定制化：**
- 可通过 `compute_metrics` 传入自定义评估函数
- 可通过 `callbacks` 添加自定义回调（如 Early Stopping）
- 可通过 `optimizers` 参数指定自定义优化器

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_2133/2641408767.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.593900
1000,0.578600
1500,0.638300
2000,0.485800
2500,0.367000
3000,0.301000
3500,0.231400
4000,0.151200
4500,0.094900


TrainOutput(global_step=4590, training_loss=0.3762191202126297, metrics={'train_runtime': 178.3236, 'train_samples_per_second': 205.693, 'train_steps_per_second': 25.74, 'total_flos': 1353042435523440.0, 'train_loss': 0.3762191202126297, 'epoch': 10.0})

**评估**

In [14]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
# predict() 方法的输出一个带有三个字段的命名元组: predictions ,label_ids 和 metrics
# metrics 字段将只包含所传递的数据集的损失,以及一些时间指标(总共花费的时间和平均预测时间)
# predictions.prediction (batch,num_class)
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

(408, 2) (408,)


{'accuracy': 0.8259803921568627, 'f1': 0.88107202680067}

在cpu上训练最后一层3个epoch，准确率67%。bert的语言潜能还没有完全适应新任务，只训练3个epoch太少了。微调还是用GPU吧。    
使用gpu训练所有参数10个epoch,准确率82%

最后把所有东西打包在一起，我们就得到了 `compute_metrics()` 函数

该函数必须接收一个 `EvalPrediction` 对象（它是一个带有 `predictions` 和 `label_ids` 字段的参数元组），并将返回一个字符串映射到浮点数的字典（字符串是返回的指标名称，而浮点数是其值）

In [15]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

为了查看模型在每个训练周期结束时的好坏，下面是我们如何使用 `compute_metrics()` 函数定义一个新的 `Trainer`

In [16]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_2133/1799240845.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training Process

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

在正式开始编写我们的训练循环之前，我们需要定义一些对象。首先是我们将用于迭代 batch 的数据加载器。但在定义这些数据加载器之前，我们需要对我们的 `tokenized_datasets` 进行一些后处理，以自己实现一些 Trainer 自动为我们处理的内容。具体来说，我们需要：
- 删除与模型不需要的列（如 `sentence1` 和 `sentence2` 列）。
- 将列名 `label` 重命名为 `labels` （因为模型默认的输入是 `labels` ）。
- 设置数据集的格式，使其返回 PyTorch 张量而不是列表。

In [18]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

定义数据加载器DataLoader

| **参数** | **类型** | **含义** |
| --- | --- | --- |
| dataset | Dataset | 要加载的数据集对象，必须是 torch.utils.data.Dataset 的实例。 |
| batch_size | int | 每个批次包含的样本数，默认是 1。 |
| shuffle | bool | 是否在每个 epoch 开始时打乱数据，默认是 False。 |
| num_workers | int | 使用多少个子进程加载数据，默认是 0（主进程加载）。 |
| drop_last | bool | 如果数据集大小不能整除 batch_size，是否丢弃最后一个不完整的批次，默认是 False。 |
| collate_fn | callable | 自定义函数，用于将多个样本组合成一个批次，默认会自动转为张量。 |
| sampler | Sampler | 自定义采样器，控制数据加载顺序（与 shuffle=True 互斥）。 |

In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## tqdm：进度条库

**tqdm** 是轻量级进度条库，用于实时显示循环进度。

**常用参数：**
- `iterable`：要迭代的对象
- `desc`：进度条前缀描述（如 "Training"）
- `total`：总迭代次数（可选）
- `unit`：迭代单位（默认 "it"）
- `leave`：完成后是否保留进度条（默认 True）

**用法示例：**
```python
from tqdm.auto import tqdm

for epoch in tqdm(range(num_epochs), desc="Epoch"):
    for batch in tqdm(dataloader, desc="Training"):
        # 训练逻辑
        pass
```

**效果：** 实时显示当前进度百分比、迭代速度、估计剩余时间等，提升用户体验。

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

## evaluate：Hugging Face 评估库

**evaluate** 库提供了 NLP 常用的评估指标。

**主要指标：**
- **分类**：`accuracy`, `f1`, `precision`, `recall`
- **相似度**：`pearsonr`, `spearmanr`
- **生成**：`bleu`, `rouge`, `meteor`
- **GLUE**：`glue`（包含多个 GLUE 任务的指标）

**使用流程：**
```python
metric = evaluate.load("accuracy")
for batch in dataloader:
    predictions = model(batch)
    metric.add_batch(predictions=predictions, references=batch["labels"])
result = metric.compute()  # 返回 {"accuracy": 0.87}
```

**优点**：
- 统一的 API，与 Hugging Face 生态无缝集成
- 自动处理多 GPU 聚合
- 广泛支持标准 NLP 指标

In [ ]:
metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8553921568627451, 'f1': 0.8977469670710572}